##  Libraries

In [2]:
import sys
sys.path.append('../')



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
import math
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

##  Load data

In [28]:
df_train = pd.read_csv("Input/clean_train.csv")
df_test = pd.read_csv("Input/clean_test.csv")

In [29]:
df_train

carat  cut  color  clarity     x     y     z  price     cut/wt  \
0       1.50    4      3        4  7.32  7.34  4.51  9.588   2.666667   
1       2.01    6      1        2  8.11  8.25  4.96  9.748   2.985075   
2       0.50    3      1        3  5.13  5.09  3.15  7.255   6.000000   
3       0.25    6      3        6  4.05  4.08  2.50  6.450  24.000000   
4       0.52    3      4        4  5.16  5.19  3.21  7.721   5.769231   
...      ...  ...    ...      ...   ...   ...   ...    ...        ...   
40450   1.04    6      6        2  6.60  6.62  3.94  8.190   5.769231   
40451   0.51    6      1        3  5.09  5.05  3.21  7.246  11.764706   
40452   1.51    3      5        4  7.37  7.33  4.60  9.277   1.986755   
40453   2.02    4      5        4  8.16  8.11  4.99  9.680   1.980198   
40454   0.50    1      3        4  5.31  5.27  2.95  7.154   2.000000   

        color/wt  clarity/wt  
0       2.000000    2.666667  
1       0.497512    0.995025  
2       2.000000    6.000000  
3      12.000000   24.000000  
4       7.692308    7.692308  
...          ...         ...  
40450   5.769231    1.923077  
40451   1.960784    5.882353  
40452   3.311258    2.649007  
40453   2.475248    1.980198  
40454   6.000000    8.000000  

[40455 rows x 11 columns]

In [30]:
y = df_train["price"]
X = df_train.drop(columns=["price"])

In [31]:
X_test = df_test

## Many estimators

In [32]:
pipe = Pipeline(steps=[('estimator', LinearRegression())])

In [33]:
# Add a dict of estimator and estimator related parameters in this list
params_grid = [{
                'estimator':[LinearRegression()],
                },
                {
                'estimator': [DecisionTreeRegressor()],
                'estimator__max_depth': [1, 2,3, None],
                'estimator__max_features': ["sqrt", "log2"],
                },
                {'estimator':[RandomForestRegressor()],
                 'estimator__n_estimators':[50,100,200,300],
                 'estimator__max_depth':[1, 2,3, None], 
                 'estimator__min_samples_split':[2,3], 
                 'estimator__min_samples_leaf':[1,2,3],
                },
                {'estimator':[KNeighborsRegressor()],
                 'estimator__n_neighbors':[3,5],
                }

              ]

In [35]:
grid = GridSearchCV(pipe, params_grid, n_jobs= -1, verbose = 2)

In [36]:
%%time
res = grid.fit(X, y)

Fitting 5 folds for each of 107 candidates, totalling 535 fits
CPU times: user 42.8 s, sys: 2.55 s, total: 45.4 s
Wall time: 17min 46s


In [37]:
grid.best_params_

{'estimator': RandomForestRegressor(min_samples_leaf=2, min_samples_split=3, n_estimators=300),
 'estimator__max_depth': None,
 'estimator__min_samples_leaf': 2,
 'estimator__min_samples_split': 3,
 'estimator__n_estimators': 300}

In [38]:
pd.DataFrame(grid.cv_results_).head()

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.058584      0.014798         0.009811        0.005125   
1       0.041028      0.008502         0.007095        0.003785   
2       0.029073      0.002484         0.005329        0.001100   
3       0.035639      0.003446         0.006484        0.001686   
4       0.037626      0.005303         0.005726        0.001803   

           param_estimator param_estimator__max_depth  \
0       LinearRegression()                        NaN   
1  DecisionTreeRegressor()                          1   
2  DecisionTreeRegressor()                          1   
3  DecisionTreeRegressor()                          2   
4  DecisionTreeRegressor()                          2   

  param_estimator__max_features param_estimator__min_samples_leaf  \
0                           NaN                               NaN   
1                          sqrt                               NaN   
2                          log2                               NaN   
3                          sqrt                               NaN   
4                          log2                               NaN   

  param_estimator__min_samples_split param_estimator__n_estimators  \
0                                NaN                           NaN   
1                                NaN                           NaN   
2                                NaN                           NaN   
3                                NaN                           NaN   
4                                NaN                           NaN   

  param_estimator__n_neighbors  \
0                          NaN   
1                          NaN   
2                          NaN   
3                          NaN   
4                          NaN   

                                              params  split0_test_score  \
0                  {'estimator': LinearRegression()}           0.963707   
1  {'estimator': DecisionTreeRegressor(), 'estima...           0.725630   
2  {'estimator': DecisionTreeRegressor(), 'estima...           0.723909   
3  {'estimator': DecisionTreeRegressor(), 'estima...           0.876379   
4  {'estimator': DecisionTreeRegressor(), 'estima...           0.813851   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           0.965991           0.936107           0.966071           0.962872   
1           0.725752           0.534320           0.725403           0.566627   
2           0.725752           0.723870           0.723752           0.723944   
3           0.878943           0.842938           0.822960           0.877745   
4           0.727794           0.813480           0.839380           0.878050   

   mean_test_score  std_test_score  rank_test_score  
0         0.958950        0.011490               29  
1         0.655546        0.086398              107  
2         0.724245        0.000756              106  
3         0.859793        0.022825               80  
4         0.814511        0.049350               81

In [39]:
mse_scorer = make_scorer(mean_squared_error)
cross_validate(estimator=RandomForestRegressor(max_depth=None,
                        min_samples_split=5,min_samples_leaf = 1, n_estimators=300), 
               X=X, y=y, scoring=mse_scorer, cv=10, verbose = 2,  n_jobs= -1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=  32.8s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.8s remaining:    0.0s


[CV] END .................................................... total time=  36.0s
[CV] END .................................................... total time=  36.3s
[CV] END .................................................... total time=  34.8s
[CV] END .................................................... total time=  35.7s
[CV] END .................................................... total time=  35.0s
[CV] END .................................................... total time=  38.6s
[CV] END .................................................... total time=  45.3s
[CV] END .................................................... total time=  37.8s
[CV] END .................................................... total time=  36.8s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  6.2min finished


{'fit_time': array([32.54558444, 35.61497903, 36.02232003, 34.49067569, 35.35845923,
        34.70017457, 38.16135931, 44.97799277, 37.44267201, 36.52488017]),
 'score_time': array([0.29696822, 0.34637904, 0.29109311, 0.29086876, 0.29285645,
        0.28945804, 0.4228704 , 0.31534529, 0.30936289, 0.31293988]),
 'test_score': array([0.00883842, 0.01120921, 0.0085263 , 0.00797318, 0.00811973,
        0.00800887, 0.00837439, 0.00799003, 0.00791031, 0.00938049])}

In [40]:
best_final = res.best_estimator_
best_final

Pipeline(steps=[('estimator',
                 RandomForestRegressor(min_samples_leaf=2, min_samples_split=3,
                                       n_estimators=300))])

In [42]:
predictedY_train2 = best_final.predict(X)
predictedY_test2 =  best_final.predict(X_test)

In [43]:
#To obtain different scores of the best model
Rsquare=res.score(X,y)
rmse=np.sqrt(mean_squared_error(y,predictedY_train2))

Rsquare, rmse

(0.9976961342567132, 0.04868594297609504)

### Test data

In [44]:
mse_scorer = make_scorer(mean_squared_error)
cross_validate(estimator=RandomForestRegressor(min_samples_leaf = 8, n_estimators=300), 
               X=X_test, y=predictedY_test2, scoring=mse_scorer, cv=10, verbose = 2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   7.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.6s remaining:    0.0s


[CV] END .................................................... total time=   8.2s
[CV] END .................................................... total time=   9.0s
[CV] END .................................................... total time=   8.7s
[CV] END .................................................... total time=   8.7s
[CV] END .................................................... total time=   8.6s
[CV] END .................................................... total time=   9.1s
[CV] END .................................................... total time=   9.3s
[CV] END .................................................... total time=   8.7s
[CV] END .................................................... total time=   8.7s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.4min finished


{'fit_time': array([7.53459239, 8.15188384, 8.88584518, 8.66435122, 8.5968349 ,
        8.52579927, 8.97603416, 9.20089889, 8.60252595, 8.62632799]),
 'score_time': array([0.0611515 , 0.06975126, 0.07236314, 0.06952047, 0.07054424,
        0.0709734 , 0.08596301, 0.06947756, 0.0693171 , 0.06966329]),
 'test_score': array([0.00425602, 0.0031726 , 0.00437477, 0.00340344, 0.00334212,
        0.00330179, 0.00402963, 0.00436526, 0.00358855, 0.00360696])}

In [46]:
comparing_test2 = pd.DataFrame(predictedY_test2, columns = ["price"])
comparing_test2.index.name = 'id'

In [ ]:
comparing_test2.to_csv("Submission_todas.csv", header = True)